In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
import os
os.chdir('C:\\Users\\aravi\\Downloads')

In [3]:
data = pd.read_csv("NSSO68.csv")

C:\Users\aravi\AppData\Local\Temp\ipykernel_26660\4031375496.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("NSSO68.csv")


In [4]:
data['state_1'].unique()

array(['GUJ', 'ORI', 'CHTSD', 'MP', 'JRKD', 'WB', 'AP', 'MH', 'D&D',
       'D&NH', 'MIZ', 'TRPR', 'MANPR', 'ASSM', 'MEG', 'NAG', 'A&N',
       'PNDCRY', 'TN', 'GOA', 'KA', 'KE', 'LKSDP', 'SKM', 'Bhr', 'UP',
       'RJ', 'ARP', 'DL', 'HR', 'Pun', 'HP', 'UT', 'Chandr', 'J$K'],
      dtype=object)

In [5]:
#Subsetting the data
subset_data = data[data['state_1'] == 'TN'][['foodtotal_q', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education', 'No_of_Meals_per_day']]
print(subset_data)

       foodtotal_q  MPCE_MRP  MPCE_URP  Age  Meals_At_Home  \
20869    25.150588   2221.27   2432.50   75           90.0   
20870    28.400400   2546.21   2305.25   54           80.0   
20871    24.595815   2027.38   1767.00   60           90.0   
20872    22.189005   1301.23   1118.25   45           90.0   
20873    19.280350   1493.25   1385.40   48           90.0   
...            ...       ...       ...  ...            ...   
45688    31.555560   1350.23   1193.00   72           90.0   
45689    24.783517   2801.60   2524.00   52           90.0   
45690    19.079244    938.68    753.75   42           90.0   
45691    22.075537   1237.01   1195.50   53           90.0   
45692    17.122327   1242.63   1687.33   45           87.0   

       Possess_ration_card  Education  No_of_Meals_per_day  
20869                  1.0        1.0                  3.0  
20870                  1.0        8.0                  3.0  
20871                  1.0        5.0                  3.0  
20872      

In [6]:
#Checking for missing values
print(subset_data['MPCE_MRP'].isna().sum())
print(subset_data['MPCE_URP'].isna().sum())
print(subset_data['Age'].isna().sum())
print(subset_data['Possess_ration_card'].isna().sum())
print(data['Education'].isna().sum())

0
0
0
0
7


In [7]:
#Creating a function to impute th emissing values with the mean of the variable
def impute_with_mean(data, columns):
    for column in columns:
        data[column].fillna(data[column].mean(), inplace=True)
    return data

In [8]:
#Imputiong the columns
columns_to_impute = ['Education', 'MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'foodtotal_q']

In [9]:
subset_data = impute_with_mean(subset_data, columns_to_impute)

In [10]:
print(subset_data.isna().sum()) 

foodtotal_q            0
MPCE_MRP               0
MPCE_URP               0
Age                    0
Meals_At_Home          0
Possess_ration_card    0
Education              0
No_of_Meals_per_day    0
dtype: int64


In [11]:
#Fitting the regression model
X = subset_data[['MPCE_MRP', 'MPCE_URP', 'Age', 'Meals_At_Home', 'Possess_ration_card', 'Education']]
X = sm.add_constant(X)  # Adds a constant term to the predictor
y = subset_data['foodtotal_q']


In [12]:
model = sm.OLS(y, X).fit()

In [13]:
#Printinf the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            foodtotal_q   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     394.2
Date:                Sun, 23 Jun 2024   Prob (F-statistic):               0.00
Time:                        21:36:18   Log-Likelihood:                -22259.
No. Observations:                6647   AIC:                         4.453e+04
Df Residuals:                    6640   BIC:                         4.458e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   7.1171    

In [14]:
#Checking for multicollinearity using Inflator Factor (VIF)
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [15]:
print(vif_data)

               feature         VIF
0                const  108.170338
1             MPCE_MRP    1.730943
2             MPCE_URP    1.562439
3                  Age    1.222880
4        Meals_At_Home    1.127673
5  Possess_ration_card    1.195742
6            Education    1.287640


In [16]:
#Extracting the coefficients from the model
coefficients = model.params

In [17]:
#Constructing the equation
equation = f"y = {coefficients[0]:.2f}"
for i in range(1, len(coefficients)):
    equation += f" + {coefficients[i]:.6f}*x{i}"
print(equation)


y = 7.12 + 0.001256*x1 + 0.000262*x2 + 0.110306*x3 + 0.124882*x4 + -5.467140*x5 + 0.156120*x6
